# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe1406198e0>
├── 'a' --> tensor([[ 0.1011,  0.8778, -0.6750],
│                   [-0.5130, -0.7542,  0.5872]])
└── 'x' --> <FastTreeValue 0x7fe140619790>
    └── 'c' --> tensor([[ 1.6943, -1.6077, -1.4818, -0.9593],
                        [-1.7618, -0.0295,  0.9675, -1.5507],
                        [-0.2878, -1.2441, -1.2887, -0.2663]])

In [4]:
t.a

tensor([[ 0.1011,  0.8778, -0.6750],
        [-0.5130, -0.7542,  0.5872]])

In [5]:
%timeit t.a

61.2 ns ± 0.0761 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe1406198e0>
├── 'a' --> tensor([[-0.5845, -1.4240,  2.0638],
│                   [ 1.0106,  0.6085,  0.6492]])
└── 'x' --> <FastTreeValue 0x7fe140619790>
    └── 'c' --> tensor([[ 1.6943, -1.6077, -1.4818, -0.9593],
                        [-1.7618, -0.0295,  0.9675, -1.5507],
                        [-0.2878, -1.2441, -1.2887, -0.2663]])

In [7]:
%timeit t.a = new_value

67.6 ns ± 0.0697 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.1011,  0.8778, -0.6750],
               [-0.5130, -0.7542,  0.5872]]),
    x: Batch(
           c: tensor([[ 1.6943, -1.6077, -1.4818, -0.9593],
                      [-1.7618, -0.0295,  0.9675, -1.5507],
                      [-0.2878, -1.2441, -1.2887, -0.2663]]),
       ),
)

In [10]:
b.a

tensor([[ 0.1011,  0.8778, -0.6750],
        [-0.5130, -0.7542,  0.5872]])

In [11]:
%timeit b.a

57.1 ns ± 0.0118 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.3266, -1.2752,  3.2098],
               [ 0.3790,  2.0649,  1.7672]]),
    x: Batch(
           c: tensor([[ 1.6943, -1.6077, -1.4818, -0.9593],
                      [-1.7618, -0.0295,  0.9675, -1.5507],
                      [-0.2878, -1.2441, -1.2887, -0.2663]]),
       ),
)

In [13]:
%timeit b.a = new_value

483 ns ± 8.37 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

838 ns ± 5.73 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 11.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 372 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 710 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe140613220>
├── 'a' --> tensor([[[ 0.1011,  0.8778, -0.6750],
│                    [-0.5130, -0.7542,  0.5872]],
│           
│                   [[ 0.1011,  0.8778, -0.6750],
│                    [-0.5130, -0.7542,  0.5872]],
│           
│                   [[ 0.1011,  0.8778, -0.6750],
│                    [-0.5130, -0.7542,  0.5872]],
│           
│                   [[ 0.1011,  0.8778, -0.6750],
│                    [-0.5130, -0.7542,  0.5872]],
│           
│                   [[ 0.1011,  0.8778, -0.6750],
│                    [-0.5130, -0.7542,  0.5872]],
│           
│                   [[ 0.1011,  0.8778, -0.6750],
│                    [-0.5130, -0.7542,  0.5872]],
│           
│                   [[ 0.1011,  0.8778, -0.6750],
│                    [-0.5130, -0.7542,  0.5872]],
│           
│                   [[ 0.1011,  0.8778, -0.6750],
│                    [-0.5130, -0.7542,  0.5872]]])
└── 'x' --> <FastTreeValue 0x7fe08a8cf850>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 24.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe09bf90fa0>
├── 'a' --> tensor([[ 0.1011,  0.8778, -0.6750],
│                   [-0.5130, -0.7542,  0.5872],
│                   [ 0.1011,  0.8778, -0.6750],
│                   [-0.5130, -0.7542,  0.5872],
│                   [ 0.1011,  0.8778, -0.6750],
│                   [-0.5130, -0.7542,  0.5872],
│                   [ 0.1011,  0.8778, -0.6750],
│                   [-0.5130, -0.7542,  0.5872],
│                   [ 0.1011,  0.8778, -0.6750],
│                   [-0.5130, -0.7542,  0.5872],
│                   [ 0.1011,  0.8778, -0.6750],
│                   [-0.5130, -0.7542,  0.5872],
│                   [ 0.1011,  0.8778, -0.6750],
│                   [-0.5130, -0.7542,  0.5872],
│                   [ 0.1011,  0.8778, -0.6750],
│                   [-0.5130, -0.7542,  0.5872]])
└── 'x' --> <FastTreeValue 0x7fe1406196a0>
    └── 'c' --> tensor([[ 1.6943, -1.6077, -1.4818, -0.9593],
                        [-1.7618, -0.0295,  0.9675, -1.5507],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 50.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.3 µs ± 37.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.6943, -1.6077, -1.4818, -0.9593],
                       [-1.7618, -0.0295,  0.9675, -1.5507],
                       [-0.2878, -1.2441, -1.2887, -0.2663]],
              
                      [[ 1.6943, -1.6077, -1.4818, -0.9593],
                       [-1.7618, -0.0295,  0.9675, -1.5507],
                       [-0.2878, -1.2441, -1.2887, -0.2663]],
              
                      [[ 1.6943, -1.6077, -1.4818, -0.9593],
                       [-1.7618, -0.0295,  0.9675, -1.5507],
                       [-0.2878, -1.2441, -1.2887, -0.2663]],
              
                      [[ 1.6943, -1.6077, -1.4818, -0.9593],
                       [-1.7618, -0.0295,  0.9675, -1.5507],
                       [-0.2878, -1.2441, -1.2887, -0.2663]],
              
                      [[ 1.6943, -1.6077, -1.4818, -0.9593],
                       [-1.7618, -0.0295,  0.9675, -1.5507],
                       [-0.2878, -1.2441, -1.2887, -0.2663]],

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.6943, -1.6077, -1.4818, -0.9593],
                      [-1.7618, -0.0295,  0.9675, -1.5507],
                      [-0.2878, -1.2441, -1.2887, -0.2663],
                      [ 1.6943, -1.6077, -1.4818, -0.9593],
                      [-1.7618, -0.0295,  0.9675, -1.5507],
                      [-0.2878, -1.2441, -1.2887, -0.2663],
                      [ 1.6943, -1.6077, -1.4818, -0.9593],
                      [-1.7618, -0.0295,  0.9675, -1.5507],
                      [-0.2878, -1.2441, -1.2887, -0.2663],
                      [ 1.6943, -1.6077, -1.4818, -0.9593],
                      [-1.7618, -0.0295,  0.9675, -1.5507],
                      [-0.2878, -1.2441, -1.2887, -0.2663],
                      [ 1.6943, -1.6077, -1.4818, -0.9593],
                      [-1.7618, -0.0295,  0.9675, -1.5507],
                      [-0.2878, -1.2441, -1.2887, -0.2663],
                      [ 1.6943, -1.6077, -1.4818, -0.9593],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 119 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 1.86 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
